In [1]:
input_dir = '/home/matteo/ctasoft/resources'
gamma_diffuse = 'gamma_20deg_0deg_run101___cta-prod3-lapalma3-2147m-LaPalma.simtel.gz'
sys_path = '/home/matteo/ctasoft/'
prepared_data_dir = 'prepared_data/output'

In [2]:
import sys
import os
sys.path.append(sys_path + "cta-benchmarks/Preparation/parameters_search")  # path contains python_file.py
from quality_parameters_function import iterate_param, create_possible_parameters, calc_mean_std
from angular_distance_function import angular_distance

/home/matteo/miniconda3/envs/cta-benchmarks/lib/python3.6/site-packages/corsikaio/subblocks/dtypes.py:20: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return np.dtype(dict(**dt))


In [3]:
"""
script to find the optimized levels of cleaning for each image of an event.
Data are discriminated by the telescope type and then stored in the appropriate .npz file in
'output/event_cam' folder.
This means that we have optimized cleaning levels for each type of telescope

"""

from ctapipe.utils.datasets import find_in_path
from ctapipe.io import event_source
from ctapipe.calib import CameraCalibrator
import numpy as np

possible_cleaning_levels = create_possible_parameters()

In [4]:
quality_param_list = []
hillas_containers = {}

data_event = []
data_LST = []
data_Nectar = []
data_Flash = []
data_SCT = []

# load events from gamma_diffuse dir
input_url = find_in_path(gamma_diffuse, input_dir)
events = event_source(input_url, max_events=10)

calibrator = CameraCalibrator()

In [5]:
prepared_data_dir = '../../../' + prepared_data_dir + '/event_cam'

if not os.path.exists(prepared_data_dir):
    os.makedirs(prepared_data_dir, exist_ok=True)

In [6]:
for number_event, event in enumerate(events):
    hillas_containers.clear()
    calibrator(event)

    for telescope_id, dl1 in event.dl1.tel.items():
        quality_param_list.clear()

        image = dl1.image
        camera = event.inst.subarray.tels[telescope_id].camera

        optimized_cleaning_level = iterate_param(possible_cleaning_levels, event, camera, image,
                                                 telescope_id, quality_param_list, hillas_containers)

        # storing for different camera types
        if camera.cam_id == 'LSTCam':
            data_LST.append(optimized_cleaning_level)
        elif camera.cam_id == 'NectarCam':
            data_Nectar.append(optimized_cleaning_level)
        elif camera.cam_id == 'FlashCam':
            data_Flash.append(optimized_cleaning_level)
        elif camera.cam_id == 'SCTCam':
            data_SCT.append(optimized_cleaning_level)

    # calculate angular separation
    separation = angular_distance(event, hillas_containers)

    # calculate mean and standard deviation for each telescope type for the current event
    mean_LST, std_LST = calc_mean_std(data_LST)
    mean_Nectar, std_Nectar = calc_mean_std(data_Nectar)
    mean_Flash, std_Flash = calc_mean_std(data_Flash)
    mean_SCT, std_SCT = calc_mean_std(data_SCT)

    # storing data event
    data_event.append((number_event, event.mc.energy, separation, event.inst.subarray.tel_coords))
    data_event.append(('LSTCam', mean_LST, std_LST))
    data_event.append(('NectarCam', mean_Nectar, std_Nectar))
    data_event.append(('FlashCam', mean_Flash, std_Flash))
    data_event.append(('SCTCam', mean_SCT, std_SCT))

# storing in compressed files all the data
np.savez_compressed(prepared_data_dir+'/event_' + gamma_diffuse + '.npz',
                    data=data_event)

np.savez_compressed(prepared_data_dir+'/LST_parameters_' + gamma_diffuse + '.npz',
                    data=data_LST)

np.savez_compressed(prepared_data_dir+'/Nectar_parameters_' + gamma_diffuse + '.npz',
                    data=data_Nectar)

np.savez_compressed(prepared_data_dir+'/Flash_parameters_' + gamma_diffuse + '.npz',
                    data=data_Flash)

np.savez_compressed(prepared_data_dir+'/SCT_parameters_' + gamma_diffuse + '.npz',
                    data=data_SCT)